In [ ]:
*** Installing dependencies ***

In [17]:
import sys
import subprocess

exit_code = subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
if exit_code:
    raise Exception("Non-zero return code when installing the dependencies, please try reinstalling them manually")

In [ ]:
*** Adding main variables ***

In [29]:
import boto3

ami_id = "ami-0f1678fc793fca458"

ec2_client = boto3.client("ec2", "eu-central-1")

In [ ]:
*** Adding AMI id as a parameter ***

In [40]:
ssm_client = boto3.client("ssm", "eu-central-1")

param_name = "/hillel-hw15/ami-id"
param_description = "The AMI id from Packer for HW15"
param_type = "String"

ami_id_str = str(ami_id)
param_value = ami_id_str

output = ssm_client.put_parameter(
    Name=param_name,
    Description=param_description,
    Value=param_value,
    Type=param_type,
    Overwrite=True,
)

print(output)

{'Version': 6, 'Tier': 'Standard', 'ResponseMetadata': {'RequestId': '265659f9-55eb-46dc-9962-3cff2d5eb142', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Thu, 27 Apr 2023 18:46:44 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '31', 'connection': 'keep-alive', 'x-amzn-requestid': '265659f9-55eb-46dc-9962-3cff2d5eb142'}, 'RetryAttempts': 0}}


In [ ]:
*** Creating security group ***

In [31]:
security_group_name = "hw15-security-group"
security_group_description = "Group for HW15 with opened port 8081"

output = ec2_client.create_security_group(
    GroupName=security_group_name,
    Description=security_group_description
)

group_id = output["GroupId"]
print(group_id)

sg-0f42cfd8599fd39d8


In [ ]:
*** Adding incoming rule for port 8081 ***

In [42]:
ec2_client.authorize_security_group_ingress(
    GroupId=group_id,
    IpPermissions=[{
        "IpProtocol": "tcp",
        "FromPort": 8081,
        "ToPort": 8081,
        "IpRanges": [{"CidrIp": "0.0.0.0/0", "Description": "HTTP to port 8081"}]
    }]
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0325d7f5cf062c787',
   'GroupId': 'sg-0f42cfd8599fd39d8',
   'GroupOwnerId': '828516503329',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 8081,
   'ToPort': 8081,
   'CidrIpv4': '0.0.0.0/0',
   'Description': 'HTTP to port 8081'}],
 'ResponseMetadata': {'RequestId': '2b149e0e-0b34-4875-b0cf-c0912eb880fc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2b149e0e-0b34-4875-b0cf-c0912eb880fc',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '780',
   'date': 'Thu, 27 Apr 2023 18:47:38 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [ ]:
*** Getting AMI image id as a parameter ***

In [43]:
ssm_parameter = ssm_client.get_parameter(Name=param_name)
ami_id = ssm_parameter["Parameter"]["Value"]

print(ami_id)

ami-0f1678fc793fca458


In [ ]:
*** Launching instance ***

In [49]:
output = ec2_client.run_instances(
  MaxCount=1,
  MinCount=1,
  InstanceType="t2.micro",
  ImageId=ami_id,
  KeyName="Main SSH Key",
  SecurityGroupIds=[
    group_id,
  ],
)

instance_ids = list(map(lambda i : i.get("InstanceId"), output.get("Instances")))
print(f"Launched instances with ids: {' '.join(instance_ids)}")

jdun = ec2_client.get_waiter("instance_running")
jdun.wait(InstanceIds=instance_ids)
print(f"Instances with ids: {' '.join(instance_ids)} are up and running")

Launched instances with ids: i-0161063e1f6fe7df8
Instances with ids: i-0161063e1f6fe7df8 are up and running


In [ ]:
*** Getting instance IP ***

In [50]:
ec2_resource = boto3.resource("ec2", "eu-central-1")
instance = ec2_resource.Instance("i-0161063e1f6fe7df8")
print(instance.public_ip_address)

3.74.160.135


In [ ]:
*** Terminate instance ***

In [51]:
ec2_client.terminate_instances(InstanceIds=instance_ids)
jdun = ec2_client.get_waiter("instance_terminated")
jdun.wait(InstanceIds=instance_ids)
print(f"Instances with ids: {' '.join(instance_ids)} are terminated")

Instances with ids: i-0161063e1f6fe7df8 are terminated
